In this notebook, we analyze our best performing ensemble model in 

In [67]:
import numpy as np
import pandas as pd
import os
from collections import Counter
from sklearn import metrics

In [68]:
TAGS = [
'mood/theme---action',
'mood/theme---adventure',
'mood/theme---advertising',
'mood/theme---background',
'mood/theme---ballad',
'mood/theme---calm',
'mood/theme---children',
'mood/theme---christmas',
'mood/theme---commercial',
'mood/theme---cool',
'mood/theme---corporate',
'mood/theme---dark',
'mood/theme---deep',
'mood/theme---documentary',
'mood/theme---drama',
'mood/theme---dramatic',
'mood/theme---dream',
'mood/theme---emotional',
'mood/theme---energetic',
'mood/theme---epic',
'mood/theme---fast',
'mood/theme---film',
'mood/theme---fun',
'mood/theme---funny',
'mood/theme---game',
'mood/theme---groovy',
'mood/theme---happy',
'mood/theme---heavy',
'mood/theme---holiday',
'mood/theme---hopeful',
'mood/theme---inspiring',
'mood/theme---love',
'mood/theme---meditative',
'mood/theme---melancholic',
'mood/theme---melodic',
'mood/theme---motivational',
'mood/theme---movie',
'mood/theme---nature',
'mood/theme---party',
'mood/theme---positive',
'mood/theme---powerful',
'mood/theme---relaxing',
'mood/theme---retro',
'mood/theme---romantic',
'mood/theme---sad',
'mood/theme---sexy',
'mood/theme---slow',
'mood/theme---soft',
'mood/theme---soundscape',
'mood/theme---space',
'mood/theme---sport',
'mood/theme---summer',
'mood/theme---trailer',
'mood/theme---travel',
'mood/theme---upbeat',
'mood/theme---uplifting']

TAGS = [tag.split('---')[-1] for tag in TAGS]

Here we define a heirarchy for the labels:

In [70]:
tag_subsets = {}
tag_subsets['positive'] = {}

tag_subsets['positive']['low'] = ['calm',
                                    'cool',
                                    'emotional',
                                    'hopeful',
                                    'meditative',
                                    'relaxing',
                                    'slow',
                                    'soft']

tag_subsets['positive']['neutral'] = ['fun',
                                        'funny',
                                        'groovy',
                                        'happy',
                                        'inspiring',
                                        'love',
                                        'motivational',

                                        'positive',
                                        'powerful',
                                        'sexy']

tag_subsets['positive']['high'] = ['energetic',
                                    'epic',
                                    'fast',
                                    'party',
                                    'romantic',
                                    'upbeat',
                                    'uplifting']



tag_subsets['negative'] = {}

tag_subsets['negative']['sad'] = ['ballad',
                                    'melancholic',
                                    'sad']

tag_subsets['negative']['other'] = ['dark',
                                    'drama',
                                    'dramatic',
                                    'heavy']



tag_subsets['misc'] = {}


tag_subsets['misc']['media'] = ['advertising',
                            'commercial',
                            'documentary',
                            'film',
                            'game',
                            'movie',
                            'sport',
                            'trailer']

tag_subsets['misc']['seasonal'] = ['christmas',
                            'holiday',
                            'summer']

tag_subsets['misc']['other'] = ['action',
                            'adventure',
                            'background',
                            'children',
                            'corporate',
                            'deep',
                            'dream',
                            'melodic',
                            'nature',
                            'retro',
                            'soundscape',
                            'space',
                            'travel']

Function to evaluate the performance of the model given an array of predictions and a ground truth array for the given label subset.

In [71]:
def evaluate(groundtruth, predictions):

    results = {}

    for average in ['macro']:
        results['ROC-AUC-' + average] = metrics.roc_auc_score(groundtruth, predictions, average=average)
        results['PR-AUC-' + average] = metrics.average_precision_score(groundtruth, predictions, average=average)

    for metric in results:
        print(metric,'=', results[metric])

Given a subset of labels, returning the index from the alphabetized tags list, TAGS.

In [72]:
def get_label_indices(tag_subset):
    
    indices = []
    for tag in tag_subset:
        
        indices.append(TAGS.index(tag))
        
    return indices

Load the predicitions and ground truth arrays.

In [73]:
best_predictions = np.load('../predictions/ensemble_predictions_all_data.npy')

ground_truth = np.load('../predictions/test_ground_truth.npy')

Evaluate the performance of the subsets:

In [74]:
print('Positive valence, low arousal:\n')
indices = get_label_indices(tag_subsets['positive']['low'])
evaluate(ground_truth[:,indices], best_predictions[:,indices])


print('\n\nPositive valence, neutral arousal:\n')
indices = get_label_indices(tag_subsets['positive']['neutral'])
evaluate(ground_truth[:,indices], best_predictions[:,indices])


print('\n\nPositive valence, high arousal:\n')
indices = get_label_indices(tag_subsets['positive']['high'])
evaluate(ground_truth[:,indices], best_predictions[:,indices])


print('\n\nNegative valence, sad synonyms:\n')
indices = get_label_indices(tag_subsets['negative']['sad'])
evaluate(ground_truth[:,indices], best_predictions[:,indices])


print('\n\nNegative valence, other labels:\n')
indices = get_label_indices(tag_subsets['negative']['other'])
evaluate(ground_truth[:,indices], best_predictions[:,indices])


print('\n\nMedia-related labels:\n')
indices = get_label_indices(tag_subsets['misc']['media'])
evaluate(ground_truth[:,indices], best_predictions[:,indices])


print('\n\nSeason/holiday labels:\n')
indices = get_label_indices(tag_subsets['misc']['seasonal'])
evaluate(ground_truth[:,indices], best_predictions[:,indices])


print('\n\nAll other labels:\n')
indices = get_label_indices(tag_subsets['misc']['other'])
evaluate(ground_truth[:,indices], best_predictions[:,indices])

Positive valence, low arousal:

ROC-AUC-macro = 0.7223092792897011
PR-AUC-macro = 0.09213266760520744


Positive valence, neutral arousal:

ROC-AUC-macro = 0.7996187124136899
PR-AUC-macro = 0.13360330464190667


Positive valence, high arousal:

ROC-AUC-macro = 0.8222218914217266
PR-AUC-macro = 0.1384584412347932


Negative valence, sad synonyms:

ROC-AUC-macro = 0.7442986610443626
PR-AUC-macro = 0.08621283887209014


Negative valence, other labels:

ROC-AUC-macro = 0.7891849712855972
PR-AUC-macro = 0.17242628691566173


Media-related labels:

ROC-AUC-macro = 0.7825324486968115
PR-AUC-macro = 0.1774384810591242


Season/holiday labels:

ROC-AUC-macro = 0.8673664253047336
PR-AUC-macro = 0.3993026561460615


All other labels:

ROC-AUC-macro = 0.7666612450900706
PR-AUC-macro = 0.18505679956028598
